In [1]:
#Import necessary packages:
import rasterio as rio 
import xarray as xr
import rio_cogeo.cogeo
import numpy as np
from scipy.interpolate import griddata

In [2]:
#Inspect source NetCDF files:
file = 'raw-data/CYGNSS/CYGNSS_test_data.nc'

In [3]:
xds = xr.open_dataset(file)

In [5]:
# Extract lat, lon, and wind_speed
lat = xds['lat'].data
lon = xds['lon'].data
wind_speed = xds['wind_speed'].data

# Create a grid to interpolate onto (choose appropriate grid resolution)
grid_lon, grid_lat = np.meshgrid(
    np.linspace(lon.min(), lon.max(), 500),
    np.linspace(lat.min(), lat.max(), 500)
)

# Interpolate the wind_speed data to the regular grid
grid_wind_speed = griddata(
    (lon, lat), wind_speed, (grid_lon, grid_lat), method='linear'
)

# Create a new xarray DataArray with the gridded data
wind_speed_grid = xr.DataArray(
    grid_wind_speed, 
    coords=[grid_lat[:, 0], grid_lon[0, :]], 
    dims=['lat', 'lon']
)

# Ensure the dataset has the proper spatial dimensions and CRS
wind_speed_grid.rio.set_spatial_dims("lon", "lat", inplace=True)
wind_speed_grid.rio.write_crs("epsg:4326", inplace=True)

# Write the gridded data to a COG file
cog_filename = "test_cog_wind_speed.tif"
wind_speed_grid.rio.to_raster(
    cog_filename,
    driver="COG",
)

In [7]:
# Validate the COG file
cog_valid = rio_cogeo.cogeo.cog_validate(cog_filename)
print(f"COG validation: {cog_valid}")

COG validation: (True, [], [])
